# Face Detection

In [2]:
!pip install google-cloud-vision

In [4]:
from google.colab import files
files.upload()

Saving face.jpg to face.jpg


{'face.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe1$\xc6Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\r\x00\x0b\x00\x02\x00\x00\x00&\x00\x00\x08\xb6\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x08\xdc\x01\x10\x00\x02\x00\x00\x00\r\x00\x00\x08\xe2\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x08\xf0\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x08\xf8\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00&\x00\x00\t\x00\x012\x00\x02\x00\x00\x00\x14\x00\x00\t&\x01;\x00\x02\x00\x00\x00\r\x00\x00\t:\x82\x98\x00\x02\x00\x00\x00\x11\x00\x00\tH\x87i\x00\x04\x00\x00\x00\x01\x00\x00\tZ\xea\x1c\x00\x07\x00\x00\x08\x0c\x00\x00\x00\xaa\x00\x00\x13\x9e\x1c\xea\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [5]:
!ls

face.jpg  sample_data  service-account.json


In [0]:
path = r'face.jpg'

In [0]:
import os, io
from google.cloud import vision
from google.cloud.vision import types

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'service-account.json'

def starter(path):

    client = vision.ImageAnnotatorClient()
 
    if path.startswith('http') or path.startswith('gs:'):
        image = types.Image()
        image.source.image_uri = path
    else:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
        image = types.Image(content=content)
 
    result = client.face_detection(image=image).face_annotations
 
    return result


In [8]:
face = starter(path)
face

[bounding_poly {
  vertices {
    x: 343
    y: 91
  }
  vertices {
    x: 721
    y: 91
  }
  vertices {
    x: 721
    y: 530
  }
  vertices {
    x: 343
    y: 530
  }
}
fd_bounding_poly {
  vertices {
    x: 397
    y: 214
  }
  vertices {
    x: 683
    y: 214
  }
  vertices {
    x: 683
    y: 501
  }
  vertices {
    x: 397
    y: 501
  }
}
landmarks {
  type: LEFT_EYE
  position {
    x: 496.0823974609375
    y: 301.9912414550781
    z: 0.005018852651119232
  }
}
landmarks {
  type: RIGHT_EYE
  position {
    x: 611.0862426757812
    y: 311.2818298339844
    z: 19.689979553222656
  }
}
landmarks {
  type: LEFT_OF_LEFT_EYEBROW
  position {
    x: 460.7804260253906
    y: 277.0518798828125
    z: 0.8669469356536865
  }
}
landmarks {
  type: RIGHT_OF_LEFT_EYEBROW
  position {
    x: 533.0555419921875
    y: 285.4776306152344
    z: -20.355241775512695
  }
}
landmarks {
  type: LEFT_OF_RIGHT_EYEBROW
  position {
    x: 587.1000366210938
    y: 289.5089111328125
    z: -11.093803405

## Face expressions

In [6]:
# Names of likelihood from google.cloud.vision.enums
likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
likelihood_name[f.joy_likelihood]

NameError: name 'f' is not defined

In [5]:
for f in face:
  print('Joy : {}'.format(likelihood_name[f.joy_likelihood]))
  print('Sorrow : {}'.format(likelihood_name[f.sorrow_likelihood]))
  print('Anger : {}'.format(likelihood_name[f.anger_likelihood]))
  print('Surprise : {}'.format(likelihood_name[f.surprise_likelihood]))


NameError: name 'face' is not defined

## Co-ordinates for face

In [0]:
from PIL import Image, ImageDraw


In [0]:
def helper_box(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)
    # Sepecify the font-family and the font-size
    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')
        # Place the confidence value/score of the detected faces above the
        # detection box in the output image
        draw.text(((face.bounding_poly.vertices)[0].x,
                   (face.bounding_poly.vertices)[0].y - 30),
                  str(format(face.detection_confidence, '.3f')) + '%',
                  fill='#FF0000')
    im.save(output_filename)

In [13]:
face

[bounding_poly {
  vertices {
    x: 343
    y: 91
  }
  vertices {
    x: 721
    y: 91
  }
  vertices {
    x: 721
    y: 530
  }
  vertices {
    x: 343
    y: 530
  }
}
fd_bounding_poly {
  vertices {
    x: 397
    y: 214
  }
  vertices {
    x: 683
    y: 214
  }
  vertices {
    x: 683
    y: 501
  }
  vertices {
    x: 397
    y: 501
  }
}
landmarks {
  type: LEFT_EYE
  position {
    x: 496.0823974609375
    y: 301.9912414550781
    z: 0.005018852651119232
  }
}
landmarks {
  type: RIGHT_EYE
  position {
    x: 611.0862426757812
    y: 311.2818298339844
    z: 19.689979553222656
  }
}
landmarks {
  type: LEFT_OF_LEFT_EYEBROW
  position {
    x: 460.7804260253906
    y: 277.0518798828125
    z: 0.8669469356536865
  }
}
landmarks {
  type: RIGHT_OF_LEFT_EYEBROW
  position {
    x: 533.0555419921875
    y: 285.4776306152344
    z: -20.355241775512695
  }
}
landmarks {
  type: LEFT_OF_RIGHT_EYEBROW
  position {
    x: 587.1000366210938
    y: 289.5089111328125
    z: -11.093803405

In [17]:
box = [(vertex.x, vertex.y)  for vertex in f.bounding_poly.vertices for f in face]
box

[(343, 91), (721, 91), (721, 530), (343, 530)]

In [0]:
helper_box(path,face,'facebox.jpg')

In [19]:
!ls

facebox.jpg  face.jpg  sample_data  service-account.json


In [0]:
from google.colab import files
files.download('facebox.jpg')